# cFos Drug Notebook

## Setup

In [ ]:
# Import packages
import os, sys, importlib
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('functionScripts')
sys.path.append('dependencies')

import initFunctions as init #import createDirs, debugReport, loadLightSheetData
import analysisFunctions #compareAnimals, drug_stats_and_changes, brainRegionTrend
import classifyFunctions as cf #sclassifySamples, reformatData
import plotFunctions as pf #totalCountsPlot
import helperFunctions as hf
import configFunctions as config

### Switches

In [ ]:
## Switch to control whether Batch 1 and Batch 2 datasets are analyzed together. Distinct folders for each style.
batchSplit = False          # Splits drugs from the first batch of data, from the second, from the 3rd. Batch 1 is labeled with 'a' (aSAL, aKET, aPSI), Batch 3 (cKET, MDMA)
splitTag  = ['a', '', 'c']  # Appended the to beginning of data from the first batch (PSI, KET, SAL -> aPSI, KET, aSAL).

# Debug
debugOutputs = False        # Saves csvs at intervals
scalingFactor = True        # Applies 1/total_cells as a scaling factor for mice in batch one.
debug_ROI = ['Dorsal nucleus raphe']

### Configuration related dictionaries and function calls

In [ ]:
switchDir = dict(batchSplit=batchSplit, splitTag=splitTag, debugOutputs=debugOutputs, scalingFactor=scalingFactor, debug_ROI=debug_ROI)

# Make directories, and add their strings to the directory dictionary.
dirDict = init.setPath_createDirs()

# General Figure Settings
config.setup_figure_settings()

# Figure 2 related setup
heatmapDict = config.return_heatmapDict()

#### Figure 3, 4, 5, 6
# Configure the classification - testing for quicker results, default for final.

# classifyDict = config.return_classifyDict_testing()   # Test with this
classifyDict = config.return_classifyDict_default()     # Full run

# Plot dict, used to coordinate outputs of classification plotting.
plotDict = config.return_plotDict()

### Set Initial Paths, Load/Generate Lightsheet df

In [ ]:
# once directories have been made, merge data across batches and with atlases, then save.
lightsheet_data = init.loadLightSheetData(dirDict, switchDir)

# Create the Drug list for reference. Sorting doesn't seem to matter atm.
drugList = list(lightsheet_data.drug.unique())
drugListActive = [x for x in drugList if 'SAL' not in x]

## Figure 1
### Head twitch response + Cell count per drug

In [ ]:
# pf.plot_headTwitchTotal(dirDict)
pf.plotTotalPerDrug(lightsheet_data, 'total_cells', dirDict)

## Figure 2 - Data Heatmaps

In [ ]:
pf.plot_data_heatmap(lightsheet_data, heatmapDict, dirDict)
# pf.distance_matrix(lightsheet_data, classifyDict, dirDict)

## Figure 3 - Drug Classifier

In [ ]:
### Figure 3b - Supervised Dim Red via LDA
pf.plotLowDimEmbed(lightsheet_data, 'count_norm', dirDict, 'LDA', classifyDict) # 'count_norm', 'total_cells','cell_density'

In [ ]:
### Figure 3c/d - Drug based Confusion Matrix + PR Curves
# importlib.reload(cf)
# importlib.reload(pf)
# importlib.reload(hf)

cf.classifySamples(lightsheet_data, classifyDict, plotDict, dirDict)

# Figure 4 - 'PSI vs' classification

### Run the classifications

In [ ]:
# Revise Plot dict below
plotDict['plot_ConfusionMatrix'] = True
plotDict['plot_PRcurve'] = True
plotDict['plot_SHAPsummary'] = True
plotDict['plot_SHAPforce'] = False
plotDict['featureCorralogram'] = False

classifyVec = ['class_Psi5MEO', 'class_PsiKet', 'class_PsiMDMA', 'class_PsiDF', 'class_PsiSSRI', 'class_DT', 'class_5HTR', 'class_5HT2A', 'class_SSRI']
     
for i in classifyVec:
    try:
        print(f"Classifying {i}")
        classifyDict['label'] = i
        cf.classifySamples(lightsheet_data, classifyDict, plotDict, dirDict)
    except Exception:
        print(f"\n Failed to classify {i}")

# Figure 4
### Setup

In [ ]:
importlib.reload(hf)
importlib.reload(config)
importlib.reload(pf)

# Figure Settings
config.setup_figure_settings()

filterByFreq = .75 * classifyDict['CV_count']

sortedNames = ['5MEO vs PSI', 'MDMA vs PSI', 'A-SSRI vs PSI', 'KET vs PSI', '6-F-DET vs PSI', '5MEO vs 6-F-DET', 'A-SSRI vs C-SSRI', 'MDMA vs PSI/5MEO', '6-F-DET vs PSI/5MEO']

sortedNames.reverse()

featureLists, comparisonNames, aucScores, meanScores, aucScrambleScores, meanScrambleScores = hf.retrieve_dict_data(dirDict, classifyDict)

sortIdx = hf.sort_comparison_idx(sortedNames, comparisonNames)

featureLists = [featureLists[i] for i in sortIdx]
comparisonNames = [comparisonNames[i] for i in sortIdx]
aucScores = [aucScores[i] for i in sortIdx]
meanScores = [meanScores[i] for i in sortIdx]
aucScrambleScores = [aucScrambleScores[i] for i in sortIdx]
meanScrambleScores = [meanScrambleScores[i] for i in sortIdx]

### Figure 4b - mean AUPRC across comparisons

In [ ]:
# Color palette
colorsList = [[100, 100, 100], [180, 180, 180]]
colorsList = np.array(colorsList)/256
importlib.reload(pf)

pf.plot_cross_model_AUC(comparisonNames, aucScores, aucScrambleScores, colorsList, dirDict)

### Figure 4c/d - Feature counts and frequencies

In [ ]:
importlib.reload(pf)

# Reverse both only once for the final plot.
# comparisonNames.reverse()
# featureLists.reverse()

pf.plot_featureCount_violin(comparisonNames, featureLists, dirDict)
# pf.plot_similarity_matrix(comparisonNames, featureLists, filterByFreq, dirDict)
# pf.plot_featureOverlap_VennDiagram(comparisonNames, featureLists, filterByFreq, dirDict)
# pf.plot_featureHeatMap(lightsheet_data, comparisonNames, featureLists, filterByFreq, dirDict)9+

# Figure 5/6 - SHAP

### PSI vs 5MEO SHAP analysis

In [ ]:
plotDict['plot_ConfusionMatrix'] = False
plotDict['plot_PRcurve'] = False
plotDict['plot_SHAPsummary'] = True
plotDict['plot_SHAPforce'] = True
importlib.reload(cf)
importlib.reload(pf)

classifyDict['label'] = 'class_Psi5MEO'

cf.classifySamples(lightsheet_data, classifyDict, plotDict, dirDict)

### PSI vs KET, A-SSRI, and MDMA analysis

In [ ]:
# Revise Plot dict below
plotDict['plot_SHAPsummary'] = True
plotDict['plot_SHAPforce'] = False

# classifyVec = ['class_PsiMDMA', 'class_PsiSSRI', 'class_PsiKet']
classifyVec = ['class_PsiKet']

for i in classifyVec:
    try:
        print(f"Classifying {i}")
        classifyDict['label'] = i
        cf.classifySamples(lightsheet_data, classifyDict, plotDict, dirDict)
    except Exception:
        print(f"\n Failed to classify {i}")